## Sistema de predicción en tiempo real basado en detección de contornos

Además del sistema básico de predicción en tiempo real, se desarrolló una versión más completa y robusta que incorpora un proceso de **detección de contornos** para localizar números dentro de la escena antes de clasificarlos.  
Este sistema replica de manera más fiel el funcionamiento de un detector de dígitos en situaciones reales.

### 🔹 Captura y preprocesamiento del video

Cada fotograma capturado por la cámara se procesa mediante los siguientes pasos:

1. **Conversión a escala de grises:**
   Se transforma el frame de BGR a escala de grises para simplificar el procesamiento.

2. **Suavizado con Gaussian Blur:**
   ```python
   blur = cv2.GaussianBlur(gray, (7, 7), 0)


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Cargar modelo
model = tf.keras.models.load_model("mnist_best_model.h5")
print("Modelo cargado.")

# Abrir cámara
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("No se pudo abrir la cámara.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Blur + umbral
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    _, thresh = cv2.threshold(
        blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # Detectar contornos
    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Filtro para evitar ruido (muy importante)
        if w < 20 or h < 20:
            continue
        if h < 40:   # requiere altura mínima
            continue

        # Crear ROI del número
        roi = thresh[y:y+h, x:x+w]

        # Preprocesar igual que MNIST
        digit = roi.copy()
        digit = cv2.resize(digit, (28, 28))
        digit = digit.astype("float32") / 255.0
        digit = np.expand_dims(digit, axis=-1)
        digit = np.expand_dims(digit, axis=0)

        # Predicción
        preds = model.predict(digit, verbose=0)[0]
        pred_class = int(np.argmax(preds))
        pred_prob = float(np.max(preds))

        # Dibujar cuadro y texto
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"{pred_class} ({pred_prob*100:.1f}%)",
                    (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0), 2)

        # Mandarlo a consola también
        #print(f"Detectado {pred_class} con {pred_prob:.2f}")

    # Mostrar ventanas
    cv2.imshow("Real-Time Digit Recognition", frame)
    cv2.imshow("Umbral", thresh)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
